# Model Server

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [3]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

In [4]:
import os
from cloudpickle import load
import numpy as np
from typing import List
from datetime import datetime
import mlrun

In [5]:
class ClassifierModel(mlrun.runtimes.MLModelServer):
    def load(self):
        """Load model from storage."""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.model.predict(feats)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
        return resp

In [6]:
# nuclio: end-code

# Test models locally and deploy

The sklearn-project generated one or more models that will be deployed in the server project `sklearn-servers`

### test locally

In [7]:
import cloudpickle as cp
models_path = '/User/ml/demos/sklearn-pipe/models'

from sklearn.datasets import load_iris
iris = load_iris()

x = iris['data'].tolist()
y = iris['target']

for model in os.listdir(models_path):
    if model.endswith(".pkl"):
        
        my_server = ClassifierModel('classifier', model_dir=os.path.join(models_path, model))
        my_server.load()

        a = my_server.predict({"instances": x})

        assert len(a)==150

## document and save

In [8]:
from mlrun import new_model_server
fn = new_model_server('sklearn-server', model_class='ClassifierModel')
fn.spec.description = "generic sklearn model server"
fn.metadata.categories = ['serving', 'ml']
fn.metadata.labels = {'author': 'yaronh', 'framework': 'sklearn'}
#print(fn.to_yaml())
fn.export()

> 2020-08-10 12:54:58,765 [info] function spec saved to path: function.yaml


##  Deploy server

In [10]:
from mlrun import mount_v3io
fn.apply(mount_v3io())
fn.set_envs({'SERVING_MODEL_iris_dataset_v1': models_path,
             'INFERENCE_STREAM': 'users/admin/tststream'})
#fn.verbose = True
address = fn.deploy(project='sk-project')

> 2020-08-10 12:55:08,975 [info] deploy started
[nuclio] 2020-08-10 12:55:10,073 (info) Build complete
[nuclio] 2020-08-10 12:55:16,173 (info) Function deploy complete
[nuclio] 2020-08-10 12:55:16,181 done updating sk-project-sklearn-server, function address: 34.202.248.16:30045


##  Test server

In [11]:
predict_url = address+"/iris_dataset_v1/predict"
my_data = '''{"instances":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
!curl {predict_url} -d '{my_data}'

[0, 2]